In [111]:
import numpy as np
from mfglib.alg.greedy_policy_given_mean_field import Greedy_Policy
from mfglib.mean_field import mean_field
from mfglib.env import Environment
from mfglib.alg.utils import _ensure_free_tensor
import torch

In [112]:
def F(x_vec):
    """
    Define your function F(x) = f(x) - x.
    Here, x_vec is the flattened version of your T x S matrix.
    """
    # For demonstration, let’s assume f(x) = A*x_vec + b,
    # where A and b are given. Replace this with your actual function.
    A = np.array([[0.5, 0.2], [0.1, 0.7]])
    b = np.array([0.1, -0.2])
    return A @ x_vec + b - x_vec  # Note the subtraction of x_vec

def compute_jacobian(x_vec, epsilon=1e-6):
    n = len(x_vec)
    J = np.zeros((n, n))
    Fx = F(x_vec)
    for l in range(n):
        x_vec_perturbed = x_vec.copy()
        x_vec_perturbed[l] += epsilon
        Fx_perturbed = F(x_vec_perturbed)
        # Forward difference approximation for the l-th column
        J[:, l] = (Fx_perturbed - Fx) / epsilon
    return J

# Example dimension: here T = 1, S = 2 for simplicity
T, S = 1, 2  # For higher dimensions, adjust accordingly
x_initial = np.array([0.5, 0.5])  # Flattened initial guess

# Compute F(x)
Fx = F(x_initial)
# Compute the Jacobian
J = compute_jacobian(x_initial)

# Solve for update: J * Delta = -F(x)
Delta = np.linalg.solve(J, -Fx)

print("Update Delta:", Delta)
print("New x:", x_initial + Delta)

Update Delta: [-0.57692308 -1.19230769]
New x: [-0.07692308 -0.69230769]


In [113]:
instance = Environment.beach_bar(n=3, bar_loc=1, T=2)
pi = _ensure_free_tensor("uniform", instance)
L = mean_field(instance, pi)
print(instance.T)
print(L.shape)
shape_record = L.shape

torch.Size([3, 3])
2
torch.Size([3, 3, 3])


In [129]:
def F(L, env_instance):
    L = L.reshape(shape_record)
    pi_br = Greedy_Policy(env_instance, L)
    mu_final = torch.sum(L, axis=2)[-1]
    mu_final /= torch.sum(mu_final)
    env_instance.update_initial_distribution(mu_final)
    L_br = mean_field(env_instance, pi_br)
    return (L_br - L).flatten()

def compute_jacobian(x_vec, env_instance, epsilon=1e-6):
    n = len(x_vec)
    J = torch.zeros((n, n))
    Fx = F(x_vec, env_instance)
    for l in range(n):
        x_vec_perturbed = x_vec.clone()
        x_vec_perturbed[l] += epsilon
        Fx_perturbed = F(x_vec_perturbed, env_instance)
        # Forward difference approximation for the l-th column
        J[:, l] = (Fx_perturbed - Fx) / epsilon
    return J

def regularize(L):
    T = L.shape[0]
    L_new = L.clone()
    for t in range(T):
        L_new[t] = L[t] + torch.min(L[t])
        L_new[t] = L_new[t] / torch.sum(L_new[t])
    return L

# Example dimension: here T = 1, S = 2 for simplicity
L_initial = mean_field(instance, pi)

# Compute F(x)
for i in range(150):
    Fx = F(L_initial.flatten(), instance)
    print(torch.max(torch.abs(Fx)))
    # Compute the Jacobian
    J = compute_jacobian(L_initial.flatten(), instance)

    # Solve for update: J * Delta = -F(x)
    Delta = np.linalg.solve(J, -Fx)
    Delta = Delta.reshape(shape_record)
    # for t in range(L.shape[0]):
    #     Delta[t] -= torch.mean(Delta[t])
    L_initial = L_initial + 0.1* Delta
    print(L_initial)
    mu = torch.sum(L_initial, axis=2)
    print(mu)

tensor(0.4086)
tensor([[[0.0756, 0.0756, 0.1011],
         [0.1488, 0.1996, 0.1488],
         [0.1009, 0.0756, 0.0756]],

        [[0.1063, 0.1063, 0.1316],
         [0.0874, 0.1387, 0.0874],
         [0.1319, 0.1063, 0.1063]],

        [[0.1088, 0.1344, 0.1088],
         [0.0823, 0.1336, 0.0823],
         [0.1088, 0.1342, 0.1088]]])
tensor([[0.2522, 0.4972, 0.2520],
        [0.3442, 0.3135, 0.3444],
        [0.3521, 0.2982, 0.3519]])
tensor(0.3664)
tensor([[[0.0680, 0.0680, 0.1164],
         [0.1339, 0.2289, 0.1339],
         [0.1154, 0.0680, 0.0680]],

        [[0.0956, 0.0956, 0.1432],
         [0.0786, 0.1739, 0.0786],
         [0.1434, 0.0956, 0.0956]],

        [[0.0979, 0.1457, 0.0979],
         [0.0740, 0.1703, 0.0740],
         [0.0979, 0.1455, 0.0979]]])
tensor([[0.2523, 0.4968, 0.2514],
        [0.3345, 0.3311, 0.3347],
        [0.3416, 0.3184, 0.3414]])
tensor(0.3297)
tensor([[[0.0612, 0.0612, 0.1303],
         [0.1205, 0.2557, 0.1205],
         [0.1293, 0.0612, 0.0612]],



In [128]:
def F(L, env_instance):
    L = L.reshape(shape_record)
    pi_br = Greedy_Policy(env_instance, L)
    mu_final = torch.sum(L, axis=2)[-1]
    mu_final /= torch.sum(mu_final)
    env_instance.update_initial_distribution(mu_final)
    L_br = mean_field(env_instance, pi_br)
    return (L_br - L).flatten()

def compute_jacobian(x_vec, env_instance, epsilon=1e-6):
    n = len(x_vec)
    J = torch.zeros((n, n))
    Fx = F(x_vec, env_instance)
    for l in range(n):
        x_vec_perturbed = x_vec.clone()
        x_vec_perturbed[l] += epsilon
        Fx_perturbed = F(x_vec_perturbed, env_instance)
        # Forward difference approximation for the l-th column
        J[:, l] = (Fx_perturbed - Fx) / epsilon
    return J

def project_to_simplex(v):
    n = v.shape[0]
    u, _ = torch.sort(v, descending=True)
    cssv = torch.cumsum(u, dim=0)
    ind = torch.arange(1, n + 1, device=v.device, dtype=v.dtype)
    cond = u + (1 - cssv) / ind
    valid = (cond > 0).nonzero(as_tuple=False).flatten()
    rho = valid[-1].item() if valid.numel() > 0 else 0
    theta = (cssv[rho] - 1) / (rho + 1)
    return torch.clamp(v - theta, min=0)

L_initial = mean_field(instance, pi)
damp = 0.1

# Compute F(x)
for i in range(500):
    Fx = F(L_initial.flatten(), instance)
    print(torch.max(torch.abs(Fx)))
    J = compute_jacobian(L_initial.flatten(), instance)
    Delta = np.linalg.solve(J, -Fx)
    L_initial = L_initial + Delta.reshape(shape_record)
    for t in range(L_initial.shape[0]):
        L_initial[t] = project_to_simplex(L_initial[t].flatten()).reshape(shape_record[1:])
    mu = torch.sum(L_initial, axis=2)
    print(mu)
    

tensor(0.4086)
tensor([[0.2513, 0.4942, 0.2545],
        [0.2471, 0.5037, 0.2491],
        [0.2531, 0.4957, 0.2512]])
tensor(0.3138)
tensor([[0.3313, 0.3375, 0.3313],
        [0.3311, 0.3401, 0.3288],
        [0.3319, 0.3363, 0.3318]])
tensor(0.3287)
tensor([[0.2511, 0.4979, 0.2511],
        [0.2518, 0.4964, 0.2518],
        [0.2511, 0.4979, 0.2511]])
tensor(0.3099)
tensor([[0.3239, 0.3348, 0.3413],
        [0.3289, 0.3264, 0.3446],
        [0.3239, 0.3351, 0.3410]])
tensor(0.3446)
tensor([[0.2510, 0.4978, 0.2512],
        [0.2518, 0.4963, 0.2520],
        [0.2510, 0.4978, 0.2512]])
tensor(0.3098)
tensor([[0.3335, 0.3389, 0.3276],
        [0.3344, 0.3380, 0.3276],
        [0.3335, 0.3388, 0.3277]])
tensor(0.3344)
tensor([[0.2511, 0.4979, 0.2510],
        [0.2518, 0.4964, 0.2518],
        [0.2511, 0.4979, 0.2510]])
tensor(0.3099)
tensor([[0.3280, 0.3387, 0.3334],
        [0.3274, 0.3413, 0.3313],
        [0.3281, 0.3386, 0.3334]])
tensor(0.3334)
tensor([[0.2510, 0.4979, 0.2511],
       

In [94]:
import torch.nn.functional as func
T=3
S=3
A=3
def F(L, env_instance):
    L = L.reshape(T, S*A)
    L = func.softmax(L, dim=-1).view(T,S,A)
    pi_br = Greedy_Policy(env_instance, L)
    mu_final = torch.sum(L, axis=2)[-1]
    mu_final /= torch.sum(mu_final)
    env_instance.update_initial_distribution(mu_final)
    L_br = mean_field(env_instance, pi_br)
    return (L_br - L).flatten()

def compute_jacobian(x_vec, env_instance, epsilon=1e-6):
    n = len(x_vec)
    J = torch.zeros((n, n))
    Fx = F(x_vec, env_instance)
    for l in range(n):
        x_vec_perturbed = x_vec.clone()
        x_vec_perturbed[l] += epsilon
        Fx_perturbed = F(x_vec_perturbed, env_instance)
        # Forward difference approximation for the l-th column
        J[:, l] = (Fx_perturbed - Fx) / epsilon
    return J

# Example dimension: here T = 1, S = 2 for simplicity
x_initial = mean_field(instance, pi)
L_softmax = x_initial.view(T, S*A)
L_softmax = func.softmax(L_softmax, dim=-1).view(T,S,A)
mu = torch.sum(L_softmax, axis=2)
print(mu)

# Compute F(x)
for i in range(100):
    Fx = F(x_initial.flatten(), instance)
    # Compute the Jacobian
    J = compute_jacobian(x_initial.flatten(), instance)

    # Solve for update: J * Delta = -F(x)
    Delta = np.linalg.solve(J, -Fx)
    L_initial = L_initial + Delta.reshape(shape_record)
    L_softmax = L_initial.view(T, S*A)
    L_softmax = func.softmax(L_softmax, dim=-1).view(T,S,A)
    mu = torch.sum(L_softmax, axis=2)
    print(mu)

tensor([[0.3337, 0.3326, 0.3337],
        [0.3364, 0.3271, 0.3364],
        [0.3367, 0.3267, 0.3367]])
tensor([[0.0000e+00, 1.0000e+00, 0.0000e+00],
        [2.9312e-03, 7.2255e-05, 9.9700e-01],
        [0.0000e+00, 1.0000e+00, 0.0000e+00]])
tensor([[0.0000e+00, 1.0000e+00, 0.0000e+00],
        [3.4558e-03, 9.1345e-05, 9.9645e-01],
        [0.0000e+00, 1.0000e+00, 0.0000e+00]])
tensor([[0.0000e+00, 1.0000e+00, 0.0000e+00],
        [4.0740e-03, 1.1547e-04, 9.9581e-01],
        [0.0000e+00, 1.0000e+00, 0.0000e+00]])
tensor([[0.0000e+00, 1.0000e+00, 0.0000e+00],
        [4.8022e-03, 1.4594e-04, 9.9505e-01],
        [0.0000e+00, 1.0000e+00, 0.0000e+00]])
tensor([[0.0000e+00, 1.0000e+00, 0.0000e+00],
        [5.6597e-03, 1.8443e-04, 9.9416e-01],
        [0.0000e+00, 1.0000e+00, 0.0000e+00]])
tensor([[0.0000e+00, 1.0000e+00, 0.0000e+00],
        [6.6693e-03, 2.3304e-04, 9.9310e-01],
        [0.0000e+00, 1.0000e+00, 0.0000e+00]])
tensor([[0.0000e+00, 1.0000e+00, 0.0000e+00],
        [7.8575e-

In [107]:
import torch
import torch.nn.functional as F
T=3
S=3
A=3
# ===== Example Mapping F =====
def F_mapping(L, env_instance):
    L = L.reshape(shape_record)
    pi_br = Greedy_Policy(env_instance, L)
    mu_final = torch.sum(L, axis=2)[-1]
    mu_final /= torch.sum(mu_final)
    env_instance.update_initial_distribution(mu_final)
    L_br = mean_field(env_instance, pi_br)
    return (L_br - L).flatten()

# ===== Define H(x) =====
def H(x, env_instance):
    x = x.view(T, S*A)
    y = F.softmax(x, dim=-1)
    # Compute H(x): the difference between F(y) and y.
    return (F_mapping(y, env_instance) - y.flatten())


def compute_jacobian(x_vec, env_instance, epsilon=1e-6):
    n = len(x_vec)
    J = torch.zeros((n, n))
    Fx = H(x_vec, env_instance)
    for l in range(n):
        x_vec_perturbed = x_vec.clone()
        x_vec_perturbed[l] += epsilon
        Fx_perturbed = H(x_vec_perturbed, env_instance)
        # Forward difference approximation for the l-th column
        J[:, l] = (Fx_perturbed - Fx) / epsilon
    return J

# ===== Newton's Method for a single example =====
def newton_fixed_point(x_init, env_instance, tol=1e-6, max_iter=20):
    # Make sure x_init is a tensor with gradients enabled.
    x = x_init.clone().detach().requires_grad_(True)
    for i in range(max_iter):
        # Evaluate H(x)
        H_val = H(x, env_instance)
        norm_H = torch.norm(H_val)
        if norm_H < tol:
            print(f"Converged at iteration {i} with ||H(x)|| = {norm_H.item():.2e}")
            break

        # Compute the Jacobian of H at x using autograd.
        # The jacobian will be of shape [n, n] if x is of length n.
        J = compute_jacobian(x, env_instance)
        
        # Solve for the Newton update: J * delta = -H(x)
        try:
            delta = torch.linalg.solve(J, -H_val)
        except RuntimeError as err:
            print("Jacobian solving failed:", err)
            break
        
        # Update x and re-enable gradient tracking.
        x = (x + delta).detach().requires_grad_(True)
    # Compute the final softmax transformation.
    y = F.softmax(x, dim=-1)
    return x, y

# # ===== Newton's Method for a Batch =====
# def newton_fixed_point_batch(x_init_batch, tol=1e-6, max_iter=20):
#     """
#     Applies Newton's method independently for each row in x_init_batch.
#     Each row is treated as an independent example corresponding to one t.
    
#     Args:
#         x_init_batch (torch.Tensor): A tensor of shape [T, n],
#             where n = S*A if the T examples are flattened.
    
#     Returns:
#         x_final: Tensor of shape [T, n] (unconstrained),
#         y_final: Tensor of shape [T, n] where each row lies on the simplex.
#     """
#     x_final_list = []
#     y_final_list = []
#     T = x_init_batch.shape[0]
#     for t in range(T):
#         x, y = newton_fixed_point(x_init_batch[t], tol=tol, max_iter=max_iter)
#         x_final_list.append(x)
#         y_final_list.append(y)
#     x_final = torch.stack(x_final_list)
#     y_final = torch.stack(y_final_list)
#     return x_final, y_final

# ===== Example usage =====
# Example dimensions: let T be the batch dimension, and let n = S*A.
x_initial = mean_field(instance, pi).flatten()

# Run Newton's method to find the fixed point in the reparameterized space.
x_final, y_final = newton_fixed_point(x_initial, instance)

print("Final unconstrained x:")
print(x_final)
print("\nFinal y = softmax(x) (each should lie in the simplex):")
print(y_final)
print("\nRow sums (should be 1):", y_final.sum(dim=1))


Jacobian solving failed: torch.linalg.solve: The solver failed because the input matrix is singular.
Final unconstrained x:
tensor([29.2426, 29.2426,  2.8677, 32.2995, 32.8334, 32.2995, 49.7834, 44.6618,
        44.6618,  0.1642,  0.1642,  0.6697,  0.2666,  2.8344,  0.2666,  1.5424,
         0.1642,  0.1642,  1.5752,  2.9141,  1.5752,  1.5749,  1.7285,  1.5749,
         1.5752,  2.6326,  1.5752], requires_grad=True)

Final y = softmax(x) (each should lie in the simplex):
tensor([1.1860e-09, 1.1860e-09, 4.1647e-21, 2.5215e-08, 4.3009e-08, 2.5215e-08,
        9.8821e-01, 5.8959e-03, 5.8959e-03, 2.7892e-22, 2.7892e-22, 4.6239e-22,
        3.0900e-22, 4.0285e-21, 3.0900e-22, 1.1067e-21, 2.7892e-22, 2.7892e-22,
        1.1436e-21, 4.3627e-21, 1.1436e-21, 1.1432e-21, 1.3331e-21, 1.1432e-21,
        1.1436e-21, 3.2922e-21, 1.1436e-21], grad_fn=<SoftmaxBackward0>)


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)